# <center>Machine Learning Project</center>

** **
## <center>*02 - Feature Selection*</center>

** **

The members of the `team` are:
- Ana Farinha - 20211514
- Francisco Capontes - 20211692
- Sofia Gomes - 20240848
- Rui Lourenço - 2021639

## <span style="color:salmon"> Table of Contents </span>

<a class="anchor" id="top"></a>


1. [Filter Methods](#1-filter-methods)<br>  
    1.1 [Univariate Variables](#11-univariate-variables)<br>  
    1.2 [Correlation Indices](#12-correlation-indices)<br>    
    1.3 [Chi-Squared](#13-chi-squared)<br><br>     
2. [Wrapper Methods](#2-wrapper-methods)<br>    
    2.1 [Logistic Regression](#21-logistic-regression)<br>    
    2.2 [Support Vector Machine](#22-support-vector-machine)<br><br>      
3. [Embedded Methods](#3-embedded-methods)<br>     
    3.1 [LassoCV](#31-lassocv)<br>  



In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# Sklearn packages
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, f1_score

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# embedded methods
from sklearn.linear_model import LassoCV
import scipy.stats as stats
from scipy.stats import chi2_contingency
from sklearn.feature_selection import RFE

import warnings
warnings.filterwarnings('ignore')

from utils import *
from utils_feature_selection import *


# No train temos de input:
- Average Weekly Wage
- Age at Injury

- Based on the new Age at Injury calcular o Birth Year

*Input `Birth Year`*

In [2]:
# Import dataset
train_df = pd.read_csv('preprocessed_data/train_data.csv', index_col="Claim Identifier")
#test_df =  pd.read_csv('preprocessed_data/test_data.csv', index_col="Claim Identifier")

In [3]:
numerical_features = [
    'Age at Injury','Average Weekly Wage','Birth Year','IME-4 Count',
    'WCIO Cause of Injury Code','WCIO Nature of Injury Code','WCIO Part Of Body Code',
    'Number of Dependents','Days_to_First_Hearing','Days_to_C2','Days_to_C3',
    'Enc County of Injury', 'Enc District Name','Enc Industry Code',
    'Medical Fee Region_I','Medical Fee Region_II','Medical Fee Region_III','Medical Fee Region_IV',
    'Enc WCIO Cause of Injury Code',
    'Nature_Injury_Code_Hash_0','Nature_Injury_Code_Hash_1','Nature_Injury_Code_Hash_2',
    'Nature_Injury_Code_Hash_3','Nature_Injury_Code_Hash_4','Nature_Injury_Code_Hash_5',
    'Nature_Injury_Code_Hash_6','Nature_Injury_Code_Hash_7','Nature_Injury_Code_Hash_8',
    'Nature_Injury_Code_Hash_9','Nature_Injury_Code_Hash_10','Nature_Injury_Code_Hash_11',
    'Nature_Injury_Code_Hash_12','Nature_Injury_Code_Hash_13','Nature_Injury_Code_Hash_14',
    'Nature_Injury_Code_Hash_15','Nature_Injury_Code_Hash_16','Nature_Injury_Code_Hash_17',
    'Nature_Injury_Code_Hash_18','Nature_Injury_Code_Hash_19',
    'Zip_Code_Hash_0','Zip_Code_Hash_1',
    'Zip_Code_Hash_2','Zip_Code_Hash_3','Zip_Code_Hash_4',
    'Zip_Code_Hash_5','Zip_Code_Hash_6','Zip_Code_Hash_7',
    'Zip_Code_Hash_8','Zip_Code_Hash_9','Zip_Code_Hash_10',
    'Zip_Code_Hash_11','Zip_Code_Hash_12','Zip_Code_Hash_13',
    'Zip_Code_Hash_14','Zip_Code_Hash_15','Zip_Code_Hash_16',
    'Zip_Code_Hash_17','Zip_Code_Hash_18','Zip_Code_Hash_19',
    'Claim Injury Type Encoded',
    'Accident_Season_Sin','Accident_Season_Cos'
]
categorical_features = [
    'Known Accident Date','Known Assembly Date','Known C-2 Date','Known C-3 Date',
    'Known First Hearing Date','Known Age at Injury','Known Birth Year','Accident Date_Year',
    'Accident Date_Month','Accident Date_Day','Accident Date_DayOfWeek','Assembly Date_Year',
    'Assembly Date_Month','Assembly Date_Day','Assembly Date_DayOfWeek',
    'C-2 Date_Year','C-2 Date_Month','C-2 Date_Day','C-2 Date_DayOfWeek',
    'C-3 Date_Year','C-3 Date_Month','C-3 Date_Day', 'C-3 Date_DayOfWeek',
    'First Hearing Date_Year','First Hearing Date_Month','First Hearing Date_Day','First Hearing Date_DayOfWeek',
    'Holiday_Accident','Weekend_Accident', 'Risk_Level','Gender_F','Gender_M'
]

In [4]:
X = train_df.drop(["Claim Injury Type Encoded"], axis = 1)
y = train_df["Claim Injury Type Encoded"]

In [5]:
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size = 0.25, stratify = y, shuffle = True)

# Input Age at Injury, Birth Year and Average Weekly Wage

In [6]:
X_train_BY = X_train["Birth Year"]
X_val_BY = X_val["Birth Year"]

In [7]:
#X_train_AD = X_train["Accident Date"]
#X_val_AD = X_val["Accident Date"]

#X_train.drop("Accident Date",axis=1,inplace=True)
#X_val.drop("Accident Date",axis=1,inplace=True)

In [8]:
#imputer = KNNImputer(n_neighbors=3)
#X_train = imputer.fit_transform(X_train)
#X_val = imputer.transform(X_train)

In [9]:
to_impute = ["Age at Injury","Average Weekly Wage"]
percent_missing = X_train[to_impute].isnull().mean()
imputation_value = percent_missing / ((1 / 0.97) - 1)
for col in to_impute:
        X_train[col].fillna(imputation_value[col], inplace=True)
        X_val[col].fillna(imputation_value[col], inplace=True)

In [10]:
X_train["Birth Year"] = X_train_BY
X_val["Birth Year"] = X_val_BY

In [11]:
# Filter the rows where 'Birth Year' is NaN, but 'Age at Injury' and 'Accident Date' are not NaN
condition = X_train['Birth Year'].isna() & X_train['Age at Injury'].notna() & X_train["Accident Date"].notna()
# Replace missing 'Birth Year' with the difference between 'Accident Date' year and 'Age at Injury'
X_train.loc[condition, 'Birth Year'] = X_train["Accident Date"].dt.year - X_train["Accident Date"].loc[condition, 'Age at Injury']

# Filter the rows where 'Birth Year' is NaN, but 'Age at Injury' and 'Accident Date' are not NaN in the test dataset
condition = X_val['Birth Year'].isna() & X_val['Age at Injury'].notna() & X_val["Accident Date"].notna()
# Replace missing 'Birth Year' with the difference between 'Accident Date' year and 'Age at Injury' where the condition is true
X_val.loc[condition, 'Birth Year'] = X_test["Accident Date"].dt.year - X_val["Accident Date"].loc[condition, 'Age at Injury']

AttributeError: Can only use .dt accessor with datetimelike values

In [ ]:
X_train.drop('Accident Date',axis=1,inplace=True)
X_val.drop('Accident Date',axis=1,inplace=True)

# Creating New Features

*Average Weekly Wage*

Relative Wage Compared to Median Wage:<br>
Calculate whether the injured worker’s wage is above or below the median wage for the dataset, it's potentially reflecting job type or socioeconomic factors.

In [ ]:
median_wage = X_train['Average Weekly Wage'].median()
X_train['Relative_Wage'] = np.where(X_train['Average Weekly Wage'] > median_wage, 1,0) #('Above Median', 'Below Median')
X_val['Relative_Wage'] = np.where(X_val['Average Weekly Wage'] > median_wage, 1,0) #('Above Median', 'Below Median')

*Financial Impact*

In [ ]:
financial_impact(X_train)
financial_impact(X_train)

__Binning:__ Group ages into categories like "young" or "senior" if such categories might capture different risk profiles.<br>

In [ ]:
age_bins = [0, 25, 40, 55, 70, 100]
age_labels = [0,1,2,3,4] #['Young', 'Mid-Age', 'Experienced', 'Senior', 'Elderly']
X_train['Age_Group'] = pd.cut(X_train['Age at Injury'], bins=age_bins, labels=age_labels)
X_val['Age_Group'] = pd.cut(X_val['Age at Injury'], bins=age_bins, labels=age_labels)

# Scaling

In [ ]:
mm = StandardScaler()
X_train[numerical_features] = mm.fit_transform(X_train[numerical_features])
X_val[numerical_features] = mm.transform(X_val[numerical_features])

# Feature Selection

In [ ]:
n_features = 

*Univariate variables*

In [ ]:
X_train.var()

*Corr Matrix*

In [ ]:
# initial correlation matrix with the respective values
corr_matrix = X_train[numerical_features].corr()

mask = np.tri(*corr_matrix.shape, k=0, dtype=bool)
# Keeps values where mask is True
corr_matrix = corr_matrix.where(mask)

# defines the figure size
fig, ax = plt.subplots(figsize=(20, 20))
# heatmap of the initial correlation matrix
l = sns.heatmap(corr_matrix, square=True, annot=True, fmt=".2f", vmax=1, vmin=-1, cmap='RdBu', ax=ax)
plt.title('Correlation Between Variables', size=14)
plt.show()

*XGBoosted RFE*

In [ ]:
XGB = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

In [ ]:
best_XGB = feature_selection_RFE(X_train,y_train,n_features,model=XGB)

In [ ]:
best_XGB

*CatBoosted RFE*

In [ ]:
CB = CatBoostClassifier(iterations=500, depth=6, learning_rate=0.1, cat_features=[])

In [ ]:
best_CB = feature_selection_RFE(X_train,y_train,n_features,model=CB)

In [ ]:
best_CB

*LogisticRegression*

In [ ]:
LR = LogisticRegression()

In [ ]:
best_LR = feature_selection_RFE(X_train,y_train,n_features,model=LR)

In [ ]:
best_LR

*Lasso*

In [ ]:
feature_selection_Lasso(X_train_temp[numerical_features],y_train_temp)

*Chi-squared test*

In [ ]:
for col in categorical_features+binary_features:
    TestIndependence(X_train[col],y_train,col,alpha=0.05)

<hr>

### Numerical Data

| Predictor | Spearman | RFE XGB| RFE DT | Lasso | What to do? (One possible way to "solve") |
| --- | --- | --- | --- |--- |---|
| Age at Injury | Keep? | Discard | Discard |Discard | Discard |
| IME-4 Count | Keep | Keep | Keep |Keep| Include in the model |
| WCIO Cause of Injury Code | Keep| Keep | Discard | Discard | Try with and without |
| WCIO Nature of Injury Code | Keep | Keep | Keep | Discard | Include in the model |
| WCIO Part Of Body Code | Keep | Keep | Discard | Discard | Try with and without |
| Number of Dependents | Keep | Discard | Discard | Discard | Discard |
| Years Past Accident | Keep | Keep | Discard | Keep | Include in the model |
| Assembly Years past Accident | Discard |Keep |Discart | Keep |  Discard | # Years Past Accident
| Industry Code | Keep |Keep |Discard | Keep |   Include in the model |
| Birth Year | Keep? |Keep |Discard | Keep |  Try with and without |
| Average Weekly Wage | Keep |Keep |Keep | Keep |  Include in the model |

<hr>

### Categorical Data

| Predictor | Chi-Square |
| --- | --- |
| Carrier Name | Keep |  
| Carrier Type | Keep |
| County of Injury| Keep|
| District Name| Keep|
| Gender | Keep |
| Medical Fee Region | Keep |
| Attorney/Representative | Keep |
| COVID-19 Indicator | Keep |
| First Hearing Date Occurred | Keep |
| C-2 Date Occurred | Keep |
| C-3 Date Occurred| Keep |
| Birth Year Occurred | Keep |
| Age at Injury Occurred | Keep |
| Accident Date Occurred | KEEP |

<hr>

In [ ]:
class_mapping = {
    0:'1. CANCELLED', 
    1:'2. NON-COMP',
    2:'3. MED ONLY', 
    3:'4. TEMPORARY',
    4:'5. PPD SCH LOSS', 
    5:'6. PPD NSL', 
    6:'7. PTD', 
    7:'8. DEATH'
}
test_encoder = LabelEncoder()
test_encoder.classes_ = np.array(list(class_mapping.values()))